In [ ]:
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model="gpt2")
set_seed(42)

# Example prompt
prompt = "The impact of artificial intelligence on healthcare"
output = generator(prompt, max_length=150, num_return_sequences=1)

print("✅ GPT-2 Generated Paragraph:\n")
print(output[0]['generated_text'])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ GPT-2 Generated Paragraph:

The impact of artificial intelligence on healthcare is certainly massive. But that will only ever change if we believe in something called AI – that is, the ability to control individuals who are smarter than us?

In the US, this argument involves an unmitigated moral failure to address the growing potential that AI will replace humans. Despite our best efforts, such a philosophy exists only in theory: this is largely a fantasy, and it exists to serve its theoretical purposes, and to serve the interests of the corporate capitalist class. We are doing our best to avoid the problem, but this is the case only by making an artificial society with many different types – with many different demographics, to varying degrees – that can be used to solve complex problems. As such


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# 📘 Longer sample paragraph for better learning
text = """
Artificial intelligence is transforming industries by automating tasks, improving efficiency, and enabling data-driven decisions.
Its applications in healthcare, education, and transportation are reshaping the way we live and work.
AI-powered tools are becoming increasingly integrated into daily life, driving innovation and new business models.
"""

# Character mapping
chars = sorted(set(text))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

# Hyperparameters
input_size = len(chars)
hidden_size = 128
seq_length = 40
lr = 0.01

# Data preparation
def create_sequences(text, seq_length):
    X, y = [], []
    for i in range(len(text) - seq_length):
        seq = text[i:i+seq_length]
        target = text[i+1:i+seq_length+1]
        X.append([char_to_ix[ch] for ch in seq])
        y.append([char_to_ix[ch] for ch in target])
    return X, y

X, y = create_sequences(text, seq_length)
X = torch.tensor(X)
y = torch.tensor(y)

# Define the LSTM model
class LSTMTextGen(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMTextGen, self).__init__()
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x, hidden):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self):
        return (torch.zeros(1, 1, hidden_size), torch.zeros(1, 1, hidden_size))

model = LSTMTextGen(input_size, hidden_size)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training the model
print("\n⏳ Training LSTM...")
for epoch in range(100):
    for i in range(len(X)):
        input_seq = X[i].unsqueeze(1)
        target_seq = y[i].unsqueeze(1)

        hidden = model.init_hidden()
        optimizer.zero_grad()
        output, _ = model(input_seq, hidden)
        loss = loss_fn(output.view(-1, input_size), target_seq.view(-1))
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Text generation function
def generate_text(model, start_str, length=500, temperature=1.0):
    model.eval()
    hidden = model.init_hidden()
    input_seq = torch.tensor([[char_to_ix[ch] for ch in start_str]]).T
    result = start_str

    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        last_char_logits = output[-1]
        probs = F.softmax(last_char_logits, dim=1).data.squeeze().pow(1 / temperature)
        probs = probs / probs.sum()
        char_ind = torch.multinomial(probs, 1)[0]
        char = ix_to_char[char_ind.item()]
        result += char
        input_seq = torch.tensor([[char_ind]])

    return result

# Generate paragraph using trained LSTM
print("\n✅ LSTM Generated Paragraph:\n")
print(generate_text(model, "Artificial", length=1000, temperature=1.0))


⏳ Training LSTM...
Epoch 0, Loss: 0.4457
Epoch 10, Loss: 0.0648
Epoch 20, Loss: 0.0287
Epoch 30, Loss: 0.0302
Epoch 40, Loss: 0.0169
Epoch 50, Loss: 0.0367
Epoch 60, Loss: 0.0329
Epoch 70, Loss: 0.0495
Epoch 80, Loss: 0.0092
Epoch 90, Loss: 0.0546

✅ LSTM Generated Paragraph:

Artificial intellligence is transportation and new business modells are becoming increasinablintransportation and new business models.
AI-powered tools are becomincreasin healthcare becoming increasinablthcare becoming increasinablinttrated into daily life, driving innovation and new business models.
AI-powered toools are becomincy, and work.
AI-powered tools are becoming increasinabling data-driven decision,shapingly integly integrated into daily life, driving innovation and new business models.
AI-powered toools areforming increasinabling data-driven decisions.
Its is is is integrated into daily life, driving innovation and new business modells.
AI-powered toools abusfoming increasin healthcare becoming increa

In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.0 MB/s eta 0:00:00


In [6]:
# Let's bring in the libraries we need for text generation and the UI
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gradio as gr

# Here's the sample text we'll use to train the model
text = """
Artificial intelligence is transforming industries by automating tasks, improving efficiency, and enabling data-driven decisions.
Its applications in healthcare, education, and transportation are reshaping the way we live and work.
AI-powered tools are becoming increasingly integrated into daily life, driving innovation and new business models.
"""

# Let's map characters to indices for encoding
chars = sorted(set(text))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

# Set up hyperparameters for the model
input_size = len(chars)
hidden_size = 128
seq_length = 40
lr = 0.01

# Function to create sequences for training
def create_sequences(text, seq_length):
    X, y = [], []
    for i in range(len(text) - seq_length):
        seq = text[i:i+seq_length]
        target = text[i+1:i+seq_length+1]
        X.append([char_to_ix[ch] for ch in seq])
        y.append([char_to_ix[ch] for ch in target])
    return X, y

# Prepare the training data
X, y = create_sequences(text, seq_length)
X = torch.tensor(X)
y = torch.tensor(y)

# Define the LSTM model for text generation
class LSTMTextGen(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMTextGen, self).__init__()
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x, hidden):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self):
        return (torch.zeros(1, 1, hidden_size), torch.zeros(1, 1, hidden_size))

# Initialize the model, loss function, and optimizer as global variables
model = LSTMTextGen(input_size, hidden_size)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Train the model using global variables
def train_model(epochs=100, progress=gr.Progress()):
    try:
        print("\n⏳ Training LSTM...")
        for epoch in range(epochs):
            for i in range(len(X)):
                input_seq = X[i].unsqueeze(1)
                target_seq = y[i].unsqueeze(1)

                hidden = model.init_hidden()
                optimizer.zero_grad()
                output, _ = model(input_seq, hidden)
                loss = loss_fn(output.view(-1, input_size), target_seq.view(-1))
                loss.backward()
                optimizer.step()
            # Update progress every 10 epochs
            if epoch % 10 == 0:
                progress((epoch + 1) / epochs, desc=f"Training... Epoch {epoch}/{epochs}")
                print(f"Epoch {epoch}, Loss: {loss.item():.4f}")
        return "Training completed successfully!"
    except Exception as e:
        return f"Error during training: {str(e)}"

# Function to generate text based on a starting string using global model
def generate_text(start_str):
    try:
        model.eval()
        hidden = model.init_hidden()
        # Prepare the input sequence from the starting string
        input_seq = torch.tensor([[char_to_ix[ch] for ch in start_str if ch in char_to_ix]]).T
        result = start_str

        # Generate new characters one by one, with a fixed length of 500 characters and fixed temperature of 1.0
        for _ in range(500):  # Fixed length
            output, hidden = model(input_seq, hidden)
            last_char_logits = output[-1]
            probs = F.softmax(last_char_logits, dim=1).data.squeeze().pow(1 / 1.0)  # Fixed temperature
            probs = probs / probs.sum()
            char_ind = torch.multinomial(probs, 1)[0]
            char = ix_to_char[char_ind.item()]
            result += char
            input_seq = torch.tensor([[char_ind]])

        return result, None
    except Exception as e:
        return None, f"Error during text generation: {str(e)}"

# Custom CSS for a clean, professional, and aesthetic look
custom_css = """
body {
    font-family: 'Arial', sans-serif;
}
.gr-button {
    border-radius: 12px !important;
    padding: 12px 24px !important;
    font-weight: 600 !important;
    transition: all 0.3s ease !important;
}
.gr-button:hover {
    opacity: 0.9 !important;
    transform: scale(1.02) !important;
}
.gr-textbox {
    border-radius: 12px !important;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1) !important;
    padding: 15px !important;
}
.card {
    background: #ffffff !important;
    border-radius: 16px !important;
    box-shadow: 0 6px 12px rgba(0, 0, 0, 0.15) !important;
    padding: 20px !important;
    margin-bottom: 20px !important;
}
.header {
    background: linear-gradient(135deg, #3b82f6, #60a5fa) !important;
    color: white !important;
    padding: 20px !important;
    border-radius: 12px !important;
    text-align: center !important;
}
"""

# Build the Gradio interface with a modern design
with gr.Blocks(theme=gr.themes.Monochrome(), css=custom_css) as interface:
    # Add a stylish header to welcome users
    gr.Markdown(
        """
        <div class="header">
            <h1>📝 Generative Text with LSTM</h1>
            <p>Generate creative text using a trained LSTM model. Provide a starting string and watch the AI write!</p>
        </div>
        """
    )

    with gr.Row():
        # Input section for starting string
        with gr.Column(scale=3):
            with gr.Group(elem_classes="card"):
                gr.Markdown("### ✍️ Input Settings")
                start_str = gr.Textbox(
                    label="Starting String",
                    placeholder="Enter a starting string (e.g., 'Artificial')",
                    value="Artificial"
                )

        # Output section for generated text
        with gr.Column(scale=2):
            with gr.Group(elem_classes="card"):
                gr.Markdown("### 📜 Generated Text")
                generated_text = gr.Textbox(
                    label="Generated Text",
                    placeholder="Generated text will appear here...",
                    lines=10,
                    max_lines=20,
                    interactive=False
                )
                status_output = gr.Textbox(
                    label="Status",
                    placeholder="Status messages will appear here...",
                    interactive=False,
                    visible=True
                )

    # Buttons for actions
    with gr.Row():
        train_button = gr.Button("Train Model", variant="secondary")
        generate_button = gr.Button("Generate Text", variant="primary")
        clear_button = gr.Button("Clear", variant="secondary")

    # Connect buttons to functions
    train_button.click(
        fn=train_model,
        inputs=[],
        outputs=[status_output]
    )
    generate_button.click(
        fn=generate_text,
        inputs=[start_str],
        outputs=[generated_text, status_output]
    )
    clear_button.click(
        fn=lambda: (None, None),
        inputs=None,
        outputs=[generated_text, status_output]
    )

# Launch the Gradio interface
if __name__ == "__main__":
    interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://52bea8b8ccef8c5ecb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
